In [ ]:
!pip install aima3

In [ ]:
import random
from aima3.agents import *
from aima3.notebook import psource

In [ ]:
class Wall(Obstacle):
  def __str__(self):
    return '⬛'
  
class Floor(Thing):
  def __str__(self):
    return '⬜'

class Dirt(Thing):
  def __str__(self):
    return '🟫'

class MyExplorer(Explorer):
  def __init__(self, program):
    self.last_cleaned = 0
    super().__init__(program)
  def __str__(self):
    d = self.direction.direction
    if d == Direction.R:
      return '⬇️'
    if d == Direction.L:
      return '⬆️'
    if d == Direction.U:
      return '⬅️'
    if d == Direction.D:
      return '➡️'


In [ ]:

class MyVacuumEnvironment(VacuumEnvironment):
    def __init__(self, n=3):
        self.n = n
        self.ended = False
        super().__init__(n, n)
        self.add_floor()
        self.add_walls()
        self.add_dirt()
      
    def __str__(self):
      things = ''
      for i in range(self.width):
        for j in range(self.height):
          things += str(self.list_things_at((i,j))[-1])
        things += '\n'

      return things

    def add_thing(self, thing, location=None):
      if not location:
        location = self.default_location()
      super().add_thing(thing, location)

    def percept(self, agent):
        # print(self.list_things_at(agent.location, Dirt))
        status = ('Dirty' if self.some_things_at(
            agent.location, Dirt) else 'Clean')
        bump = ('Bump' if agent.bump else'None')
        dirt_nearby = self.thing_in_three_directions(agent, Wall)

        return (status, bump, dirt_nearby)

    def thing_in_three_directions(self, agent, tclass):
      location = agent.location

      if self.some_things_at(agent.direction.move_forward(location), Dirt):
        return False

      return any([
            self.some_things_at(Direction(d).move_forward(location), Dirt)
            for d in ([Direction.R, Direction.L, Direction.U, Direction.D]) 
      ])
    
    def add_dirt(self):
      dirts = 0
      for i in range(self.width):
        for j in range(self.height):
          if not self.some_things_at((i,j), Wall) and random.choice([True, True, False]):
            self.add_thing(Dirt(), (i, j))

    def add_floor(self):
      for i in range(self.width):
        for j in range(self.height):
          self.add_thing(Floor(), (i, j))

    def add_walls(self):
      for x in range(self.width):
          self.add_thing(Wall(), (x, 0))
          self.add_thing(Wall(), (x, self.height - 1))
      for y in range(self.height):
          self.add_thing(Wall(), (0, y))
          self.add_thing(Wall(), (self.width - 1, y))
      for z in range(random.randrange(2, self.width)):
        self.add_thing(Wall(), (random.randrange(1, self.width - 2), random.randrange(1, self.height - 2)))
    
    def execute_action(self, agent, action):
        if action == 'Suck':
          agent.last_cleaned = 0  
          dirt_list = self.list_things_at(agent.location, Dirt)
          if dirt_list != []:
            dirt = dirt_list[0]
            agent.performance += 100
            self.delete_thing(dirt)
        else:
          if action == 'Forward' and not agent.bump:
            agent.last_cleaned += 1
          super().execute_action(agent, action)

        if action != 'NoOp':
          agent.performance -= 1

    def default_location(self):
      while True:
        loc = (random.choice(range(1, self.width-1)), random.choice(range(1, self.height - 1)))
        if not self.some_things_at(loc, Wall):
          print(loc)
          return loc



In [ ]:
from IPython.core.display import clear_output
import time

# Grid size
N = 7

def VacuumExplorerProgram(n):
  def program(percept):
    status, bump, dirt_nearby = percept
    if bump != 'None':
      return random.choice(['TurnRight', 'TurnLeft'])
    if status == 'Dirty':
      return 'Suck'
    if dirt_nearby: 
      return 'TurnRight'
  
    return 'Forward'
    
  return MyExplorer(program)

simple_reflex_agent = VacuumExplorerProgram(N)

vacuum_env = MyVacuumEnvironment(N)

vacuum_env.add_thing(simple_reflex_agent)

# stop after 10 steps without cleaning
while simple_reflex_agent.last_cleaned < 10:
# while True:
  clear_output(wait=True)
  print(vacuum_env)
  vacuum_env.step()
  time.sleep(0.4)





⬛⬛⬛⬛⬛⬛⬛
⬛🟫⬜⬜⬛⬇️⬛
⬛🟫⬜🟫⬛⬜⬛
⬛⬜⬜⬜⬜⬜⬛
⬛⬜⬜⬜⬜⬜⬛
⬛⬜⬜⬜⬜⬜⬛
⬛⬛⬛⬛⬛⬛⬛



In [ ]:
class AgentStatus:
	RUNNING = 0
	FINISHED = 1
	FAILED = 2

class Agent:
	def __init__(self, program):
		self.program = program
		self.status = AgentStatus.RUNNING
		self.start = 0
		self.location = 0
		self.fuel = 0
		self.performance = 0
	
	def __str__(self):
		return f"{self.location} - {self.fuel}"

class PetrolPump:
	def __init__(self, litres, next_distance):
		self.litres = litres
		self.next_distance = next_distance

class PetrolCircleEnvironment:
	def __init__(self):
		self.is_done = False
		self.trucks = []
		self.status = [
			PetrolPump(1, 5),
			PetrolPump(10, 3),
			PetrolPump(7, 5),
			PetrolPump(3, 4),
			PetrolPump(3, 6)
		]

	def percept(self, agent):
		station = self.status[agent.location]
		return (agent.fuel, agent.location, station.litres, station.next_distance)

	def add_truck(self, agent, location=0):
		agent.location = location
		self.trucks.append(agent)
		environment.is_done = False
	
	def move_forward(self, agent):
		if agent.performance == len(self.status) - 1:
			agent.status = AgentStatus.FINISHED
			environment.is_done = True
			return

		station = self.status[agent.location]
		agent.location = (agent.location+1) % len(self.status)
		agent.fuel += station.litres - station.next_distance 
		agent.performance += 1

	def step(self):
		for agent in self.trucks:
			if agent.status == AgentStatus.RUNNING:
				print(agent, self.status[agent.location].litres, self.status[agent.location].next_distance)
				if agent.program(self.percept(agent)):
					self.move_forward(agent)
				else:
					agent.status = AgentStatus.FAILED
					self.is_done = True

In [ ]:
def program(percept):
	fuel, _, litres, next_dist = percept
	if fuel + litres >= next_dist:
		return True

	return False

agent = Agent(program)

environment = PetrolCircleEnvironment()

for i in range(len(environment.status)):
	print('')
	if agent.status == AgentStatus.FINISHED:
		break
	agent = Agent(program)
	environment.add_truck(agent, i)
	while not environment.is_done:
		environment.step()
